In [1]:
import datasets

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_dataset
dataset = load_dataset("ai4privacy/pii-masking-300k")

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
        num_rows: 177677
    })
    validation: Dataset({
        features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
        num_rows: 47728
    })
})

In [4]:
input_labels = dataset['train'][:29908]['mbert_text_tokens']
target_labels = dataset['train'][:29908]['mbert_bio_labels']
document_ids = dataset['train'][:29908]['id']
full_texts = dataset['train'][:29908]['source_text']
span_labels = dataset['train'][:29908]['span_labels']

train_dataset = [{'id':document_ids[i][:-1], 'ner_tags': target_labels[i], 'tokens': input_labels[i], 'full_text': full_texts[i], 'span_labels': span_labels[i]} for i in range(29908)]


In [7]:
all_labels = ['B-STREET',
 'B-CITY',
 'I-DATE',
 'B-PASS',
 'I-CITY',
 'B-TIME',
 'B-EMAIL',
 'I-DRIVERLICENSE',
 'I-POSTCODE',
 'I-BOD',
 'B-USERNAME',
 'B-BOD',
 'B-COUNTRY',
 'B-SECADDRESS',
 'I-GIVENNAME2',
 'B-IDCARD',
 'I-SOCIALNUMBER',
 'I-PASSPORT',
 'B-IP',
 'O',
 'B-LASTNAME2',
 'B-TEL',
 'B-SOCIALNUMBER',
 'I-TIME',
 'B-BUILDING',
 'B-LASTNAME1',
 'B-PASSPORT',
 'I-TITLE',
 'I-SEX',
 'I-STREET',
 'B-STATE',
 'I-STATE',
 'B-TITLE',
 'I-GIVENNAME1',
 'B-DATE',
 'B-GEOCOORD',
 'I-IDCARD',
 'I-TEL',
 'B-GIVENNAME2',
 'B-POSTCODE',
 'I-LASTNAME2',
 'B-DRIVERLICENSE',
 'I-LASTNAME3',
 'I-GEOCOORD',
 'I-COUNTRY',
 'I-EMAIL',
 'I-PASS',
 'B-SEX',
 'B-LASTNAME3',
 'I-USERNAME',
 'I-BUILDING',
 'I-IP',
 'B-GIVENNAME1',
 'I-LASTNAME1',
 'I-SECADDRESS',
 'B-CARDISSUER',
 'I-CARDISSUER']
id2label = {i: l for i, l in enumerate(all_labels)}
label2id = {v: k for k, v in id2label.items()}
target = [l for l in all_labels if l != "O"]

In [8]:
from transformers import AutoTokenizer

deberta_tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v2-xlarge")
mbert_tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-multilingual-cased")

ModuleNotFoundError: No module named 'transformers'

In [ ]:
span_labels[0]

In [ ]:
assert True == False

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["full_text"], truncation=True, is_split_into_words=False)
    print(len(tokenized_inputs['input_ids']))
    print(examples['ner_tags'])
    print(tokenizer.convert_ids_to_tokens(tokenized_inputs['input_ids']))
    labels = []
    previous_word_idx = None
    word_ids = tokenized_inputs.word_ids()
    print(len(word_ids))
    print(word_ids)
    for i, label in enumerate(word_ids):
        word_idx = label
        if word_idx is None:
            labels.append(-100)
        elif word_idx != previous_word_idx:  
            labels.append(label2id[examples['ner_tags'][label]])
        else:
            labels.append(-100)
        previous_word_idx = word_idx

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
def replace_labels(inputstring:str, mapping:list):
    for item in mapping:
        inputstring = inputstring[:item[0]] + item[2] + inputstring[item[1]:]
    return inputstring

In [ ]:
tokenized_train_dataset = []
from ast import literal_eval
for item in train_dataset: 
    label_string = replace_labels(item['full_text'], literal_eval(item['span_labels']))
    print(item['full_text'])
    print(label_string)
    tokenized_inputs = deberta_tokenizer(item["full_text"], truncation=True, is_split_into_words=False)
    label_tokenized_inputs = deberta_tokenizer(label_string, truncation=True, is_split_into_words=False)
    print(len(tokenized_inputs['input_ids']))
    print(deberta_tokenizer.convert_ids_to_tokens(tokenized_inputs['input_ids']))
    print(len(label_tokenized_inputs['input_ids']))
    print(deberta_tokenizer.convert_ids_to_tokens(label_tokenized_inputs['input_ids']))
    break

Subject: Group Messaging for Admissions Process

Good morning, everyone,

I hope this message finds you well. As we continue our admissions processes, I would like to update you on the latest developments and key information. Please find below the timeline for our upcoming meetings:

- wynqvrh053 - Meeting at 10:20am
- luka.burg - Meeting at 21
- qahil.wittauer - Meeting at quarter past 13
- gholamhossein.ruschke - Meeting at 9:47 PM
- pdmjrsyoz1460 
Subject: Group Messaging for Admissions Process

Good morning, everyone,

I hope this message finds you well. As we continue our admissions processes, I would like to update you on the latest developments and key information. Please find below the timeline for our upcoming meetings:

- USERNAME - Meeting at TIME
- USERNAME - Meeting at TIME
- USERNAME - Meeting at TIME
- USERNAME - Meeting at TIME
- USERNAME 
112
['[CLS]', '▁Subject', ':', '▁Group', '▁Messaging', '▁for', '▁Admissions', '▁Process', '▁Good', '▁morning', ',', '▁everyone', ','

In [ ]:
import evaluate
import numpy as np

seqeval = evaluate.load("seqeval")

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [all_labels[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [all_labels[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "microsoft/deberta-v2-xlarge", num_labels=len(all_labels), id2label=id2label, label2id=label2id
)

NameError: name 'all_labels' is not defined